In [2]:
class MultiToSingleLineString(Shapefile):
    def __init__(self, datasetNumber, inputFolder, outputFolder):
        super().__init__(datasetNumber, inputFolder, outputFolder)

    def multiToSingleLineString(self):

        self.read_shapefile("Pipes")
        self.add_unique_id_attribute("Pipes")

        multiLineNumber = 0
        count = 0
        for index, row in self.gdfPipes.iterrows():
            
            geom = row['geometry']
            
            if geom != None:
                print(geom.geom_type)

                if geom.geom_type == "MultiLineString":
                    
                    count = count + 1
            
                if geom.geom_type == 'MultiLineString' and len(geom.geoms) > 1:

                    print(f" the id of the pipe is {row['id']}")

                    multiLineNumber = multiLineNumber + 1
                    
                    geom_combined = self.combine_connected_line_strings(geom)  

                    #print(geom)
                    pipeId= row['id']

                    if geom_combined != None:
                    
                        # Create a LineString geometry from the list of points
                        new_line_string = LineString(geom_combined)

                        #print(new_line_string)

                        # Update the 'geometry' column with the new LineString
                        self.gdfPipes.at[index, 'geometry'] = new_line_string

            # Save the modified GeoDataFrame to a new shapefile if needed

        print(f'The number of multiLineString objects is {count}')
        path = 'Data SIG/'+self.dataset+'/'+self.output_folder+'/Pipes.shp'
        print(f'The pipe\'s database is saved in {path}' )
        self.gdfPipes.to_file(path)

    def combine_connected_line_strings(self, multiline):
        if not isinstance(multiline, MultiLineString):
            raise ValueError("Input must be a MultiLineString")
        print(multiline)
        line_strings = []
        for line in multiline.geoms:
            line_strings.append([list(coord) for coord in line.coords])

        #print(line_strings)
            
        print(f'line string list : {line_strings}')

        list_corr = []

        for index, element in enumerate(line_strings):
            first = element[0]
            last = element[-1]
            for i in range(0, len(line_strings)):
                if index != i:
                    if last == line_strings[i][0]:
                        list_corr.append((index,i))

                    if first == line_strings[i][0]:
                        list_corr.append((index,i))

        for i in range(0, len(list_corr)):
            for j in range(0, len(list_corr)-1):
                if i != j:
                    if list_corr[i][0] == list_corr[j][0]:
                        for element in list_corr:
                            key, value = element
                            if list_corr[i][1] == key:
                                list_corr.pop(i)
                                break
                            elif list_corr[j][1] == key:
                                list_corr.pop(j)
                                break
                            else:
                                continue;

        print(f'List corr : {list_corr}')


        if len(list_corr) != len(line_strings)-1:
            print(f'the multi line string {line_strings}, is not fully connected.')
            return None
        else:
            extremity = self.find_unique_numbers_left(list_corr)
            #print(f'extremity {extremity}')
            second= self.get_second_number_matching(extremity, list_corr)
            #print(f'second {second}')
            listt = []
            listt.append(second)
            geom = []

            l = len(line_strings)
            #print(f'length is {l})')
            
            

            for i in range(0, len(line_strings[extremity[0]])):
                geom.append(line_strings[extremity[0]][i])

            for i in range(1, len(line_strings[listt[0]])):
                geom.append(line_strings[listt[0]][i])

            for tup in list_corr:
                second = self.get_second_number_matching(listt, list_corr)
                if second != None:
                    for i in range(1, len(line_strings[second])):
                        geom.append(line_strings[second][i])
                listt = []
                listt.append(second)
            
            print(geom)
            return geom
    
    def find_unique_numbers_left(self, tuples):
        # Create a dictionary to count the occurrences of each number on the left side
        left_number_counts = {}

        # Create a set to keep track of numbers that appear on the right side
        right_numbers = set()

        # Iterate through the tuples and count the occurrences of each number on the left side
        for tuple_ in tuples:
            left_number = tuple_[0]
            right_number = tuple_[1]

            if left_number in left_number_counts:
                left_number_counts[left_number] += 1
            else:
                left_number_counts[left_number] = 1

            # Mark the right number as seen
            right_numbers.add(right_number)

        # Find the numbers from the left side that occur only once and do not repeat on the right side
        unique_numbers = [number for number, count in left_number_counts.items() if count == 1 and number not in right_numbers]

        return unique_numbers

    def get_second_number_matching(self, input_number, tuples):
        for tup in tuples:
            if len(tup) >= 2 and tup[0] == input_number[0]:
                return tup[1]
        return None  # Return None if no matching tuple is found

NameError: name 'Shapefile' is not defined